In [14]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import cross_val_score
#sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
df_train = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/other_datasets/kaggle_housing_train.csv')
labels = df_train['SalePrice']
df_train = df_train.drop(['SalePrice'], 1)
df_train.shape

(1460, 80)

In [4]:
df_test = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/other_datasets/kaggle_housing_test.csv')
ids = df_test[['Id']]
df_test.shape

(1459, 80)

In [5]:
#Average is a special type of munging and needs to be done first
for i in [df_train, df_test]:
    avg_cat = ['GarageYrBlt']
    for j in avg_cat: #Average Categories
        avg = round(np.mean(i[j][i[j].isnull() == False]), 0)
        i[j] = i[j].map(lambda x: avg if pd.isnull(x) == True else x) #Turn NaN to avg

In [6]:
for i in [df_train, df_test]:
    reg_cat = i.select_dtypes(include=['float64', 'int64'])
    for j in reg_cat.columns: #Regression Categories
        i[j] = i[j].map(lambda x: 0 if pd.isnull(x) == True else x) #Turn NaN to 0

In [7]:
for i in [df_train, df_test]:
    cat_cat = i.select_dtypes(include=['object'])
    for j in cat_cat.columns: #Categorical Categories
        i[j] = i[j].map(lambda x: 'None' if pd.isnull(x) == True else x) #Turn NaN to 'None'

In [8]:
#Combine train and test datasets for processing
df_concat = pd.concat([df_train, df_test])

In [9]:
h_dum = pd.get_dummies(df_concat)
#h_train_dum = pd.get_dummies(df_train)
#h_test_dum = pd.get_dummies(df_test)
#print h_train_dum.shape
#print h_test_dum.shape
print h_dum.shape

(2919, 311)


In [10]:
X_train = h_dum.drop(['Id'], 1)[:1460]
X_test = h_dum.drop(['Id'], 1)[1460:]
y_train = labels

In [32]:
from sklearn.linear_model import LinearRegression
start_time = time.time()
LR = LinearRegression(n_jobs=-1).fit(X_train, y_train)
scores = cross_val_score(LR, X_train, y_train, cv=10, n_jobs=-1)
#print scores
print np.mean(scores), '= avg_r2'
print round(time.time() - start_time, 2), 'seconds'

0.592721193484 = avg_r2
1.39 seconds


In [35]:
from sklearn.linear_model import RidgeCV
start_time = time.time()
R = RidgeCV().fit(X_train, y_train)
scores = cross_val_score(R, X_train, y_train, cv=10, n_jobs=-1)
#print scores
print np.mean(scores), '= avg_r2'
print round(time.time() - start_time, 2), 'seconds'

0.847145142623 = avg_r2
1.77 seconds


In [37]:
from sklearn.linear_model import LassoCV
start_time = time.time()
L = LassoCV().fit(X_train, y_train)
scores = cross_val_score(L, X_train, y_train, cv=10, n_jobs=-1)
#print scores
print np.mean(scores), '= avg_r2'
print round(time.time() - start_time, 2), 'seconds'

0.715173134136 = avg_r2
2.52 seconds


In [30]:
from sklearn.linear_model import ElasticNet
start_time = time.time()
EN = ElasticNet().fit(X_train, y_train)
scores = cross_val_score(EN, X_train, y_train, cv=10, n_jobs=-1)
#print scores
print np.mean(scores), '= avg_r2'
print round(time.time() - start_time, 2), 'seconds'

0.815703731623 = avg_r2
6.95 seconds


In [25]:
from sklearn.ensemble import RandomForestRegressor
start_time = time.time()
###38 for n_jobs = -1 vs 75 for n_jobs = 1. Big difference. with n_estimators = 200
RFR = RandomForestRegressor(n_estimators=200, verbose=0, n_jobs=-1).fit(X_train, y_train)
scores = cross_val_score(RFR, X_train, y_train, cv=10)
#print scores
print np.mean(scores), '= avg_r2'
print round(time.time() - start_time, 2), 'seconds'

0.862981188994 = avg_r2
44.41 seconds


In [16]:
from sklearn.ensemble import AdaBoostRegressor
start_time = time.time()
###
ADA = AdaBoostRegressor().fit(X_train, y_train)
scores = cross_val_score(ADA, X_train, y_train, cv=10, verbose=1)
#print scores
print np.mean(scores), '= avg_r2'
print round(time.time() - start_time, 2), 'seconds'

0.795286182525 = avg_r2
12.23 seconds


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   10.9s finished


In [360]:
predictions = pd.DataFrame(RFR.predict(X_test))
predictions.columns = ['SalePrice']
###
predictions.insert(0, 'Id', ids)
#predictions.set_index('Id', inplace=True)
#predictions.reset_index(inplace=True)
predictions.head()

,Id,SalePrice
0,1461,126829.000
1,1462,153053.615
2,1463,181559.800
3,1464,183638.425
4,1465,199969.590


In [361]:
predictions.to_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/other_datasets/kaggle_house_submission.csv', header=True, index=False)

In [46]:
#ARCHIVE: WHAT COLUMNS NEED WHAT EDA

#LotFrontage - 0 - 
#Alley - None - 
#MasVnrType - None - 
#MasVnrArea - 0 - 
#BsmtQual - None - 
#BsmtCond - None - 
#BsmtExposure - None - 
#BsmtFinType1 - None - 
#BsmtFinType2 - None -
#Electrical - None - 
#FireplaceQu - None - 
#GarageType - None - 
#GarageYrBlt - Cust Avg Func - 
#GarageFinish - None - 
#GarageQual - None - 
#GarageCond - None - 
#PoolQC - None - 
#Fence - None - 
#MiscFeature - None - 
#
###
#Utilities - None
#Exterior1st - None
#Exterior2nd - None
#BsmtFinSF1 - 0
#BsmtFinSF2 - 0
#BsmtUnfSF - 0
#TotalBsmtSF - 0
#BsmtFullBath - 0
#BsmtHalfBath - 0
#KitchenQual - None
#Functional - None
#GarageCars - 0
#GarageArea - 0
#SaleType - None


#h_test['SaleType'].unique()
#df['LotFrontage'].value_counts()
#df['MasVnrArea'].describe()
#h_test['BsmtFinSF1'][h_test['BsmtFinSF1'].isnull() == True]